# Example full chip design

In [1]:
%load_ext autoreload
%autoreload 2

*Make sure to have the right kernel selected!*

In [2]:
import time
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

In [3]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

from collections import OrderedDict

from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.qubits.transmon_sqnl import TransmonPocket_sqnl
from qiskit_metal.qlibrary.lumped.cap_n_interdigital_sqnl import CapNInterdigital_sqnl

In [4]:
# Parameter

Q1_ind = '14nH'#['11nH', '10.35nH', '9.789nH', '9.34nH']
Q1_sweep = ['11nH', '10.45nH', '9.94nH', '9.46nH']
Q2_ind = '11nH'#['10.35nH', '9.789nH', '9.34nH', '9.789nH']
Q2_sweep = ['10.35nH', '9.789nH', '9.34nH', '9.789nH']
Q3_ind = '14.5nH'
Bus_length = '7mm'

In [5]:
design = metal.designs.DesignPlanar()

gui = metal.MetalGUI(design)

In [6]:
design.overwrite_enabled = True
# design.chips.main

In [7]:
# design.chips

In [8]:
design.chips.main.size.size_x = '10mm'
design.chips.main.size.size_y = '10mm'

# Qubits

In [9]:
# TransmonPocket_sqnl.get_template_options(design)

In [10]:
options_1 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    connection_pads=dict(
        readout = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '645um',
                       ground_spacing='10um',
                       claw_width='40um',
                       claw_gap='125um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '4'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
       # bus_tmp = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
       #                claw_length='30um',
       #                ground_spacing='10um',
       #                claw_width='20um',
       #                claw_gap='20um',
       #                claw_cpw_length='40um',
       #                claw_cpw_width='20um',
       #                connector_location=
       #                '3'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
       #                ) ,
        bus_12 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       claw_length='30um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='20um',
                       claw_cpw_length='40um',
                       claw_cpw_width='20um',
                       connector_location=
                       '2'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       )
    ))

options_2 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    connection_pads=dict(
        readout = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '645um',
                       ground_spacing='10um',
                       claw_width='40um',
                       claw_gap='125um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '1'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        bus_12 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       claw_length='30um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='20um',
                       claw_cpw_length='40um',
                       claw_cpw_width='20um',
                       connector_location=
                       '3'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ),
        
    ))

In [11]:
q_1 = TransmonPocket_sqnl(design,'Q_1', options = dict(
        pos_x='0.8mm', 
        pos_y='0mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance = Q1_ind,
        **options_1))

q_2 = TransmonPocket_sqnl(design,'Q_2', options = dict(
        pos_x='0.8mm', 
        pos_y='-3.390mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance = Q2_ind,
        **options_2))


In [12]:
gui.rebuild()
gui.autoscale()

# Bus resonators

In [13]:
def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [14]:
bus_12_jogs = OrderedDict()
bus_12_jogs[0] = ["L", '3400um']

bus_12 = RouteMeander(design,'Bus_12', options = dict(hfss_wire_bonds = True, 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_1',
                                                    pin='bus_12'),
                                                end_pin=Dict(
                                                    component='Q_2',
                                                    pin='bus_12')
                                            ),
                                            trace_width = '20um',
                                            lead=Dict(
                                                start_straight='1000um',
                                                end_straight = '200um',
                                                start_jogged_extension = bus_12_jogs,
                                            ),
                                            meander=Dict(
                                                asymmetry = '00um',
                                                spacing = '200um'),
                                            fillet = "99um",
                                            total_length = Bus_length))




gui.rebuild()
gui.autoscale()

# Readout Resoantors

In [15]:
launch_TL_read = LaunchpadWirebond(design, 
                                   'Launch_Q_Read', 
                                   options = dict(
                                       pos_x = '8mm', 
                                       pos_y ='-1.648mm', 
                                       orientation = '180',
                                       trace_gap = '18um',
                                       trace_width = '30um'
                                   ))
gui.rebuild()
gui.autoscale()

In [16]:
CapNInterdigital.get_template_options(design)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'north_width': '10um',
 'north_gap': '6um',
 'south_width': '10um',
 'south_gap': '6um',
 'cap_width': '10um',
 'cap_gap': '6um',
 'cap_gap_ground': '6um',
 'finger_length': '20um',
 'finger_count': '5',
 'cap_distance': '50um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'aedt_q3d_wire_bonds': False,
 'aedt_hfss_wire_bonds': False}

In [17]:
highC_PF_TL = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL', 
                               options = dict(
                                   pos_x = '7.5mm',
                                   pos_y = '-1.648mm',
                                   orientation = '-90',
                                   north_width = '30um', 
                                   north_gap   = '18um', 
                                   south_width = '30um', 
                                   south_gap   = '18um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))
gui.rebuild()
gui.autoscale()

In [18]:
otg_PF = OpenToGround(design, 'otg_PF', options = dict(pos_x='-60um',
                                                       pos_y='-1648um',
                                                       width='30um',
                                                       gap='18um',
                                                       termination_gap='18um',
                                                       orientation='180'
                                                      ))
gui.rebuild()
gui.autoscale()

In [19]:
option_PF = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL',
                     pin='south_end'),
                 end_pin=Dict(
                     component='otg_PF',
                     pin='open')),
              trace_width='30um',
              trace_gap='18um',qgeometry_types='poly'
             )
option_TL = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL',
                     pin='north_end'),
                 end_pin=Dict(
                     component='Launch_Q_Read',
                     pin='tie')),
              trace_width='30um',
              trace_gap='18um',qgeometry_types='poly'
             )
PF = RouteStraight(design, 'PF', options=option_PF)
TL = RouteStraight(design, 'TL', options=option_TL)

gui.rebuild()
gui.autoscale()

In [20]:
launch_TL_read.pins

{'tie': {'points': array([[ 7.975, -1.663],
         [ 7.975, -1.633]]),
  'middle': array([ 7.975, -1.648]),
  'normal': array([-1.,  0.]),
  'tangent': array([0., 1.]),
  'width': 0.03,
  'gap': 0.018,
  'chip': 'main',
  'parent_name': 4,
  'net_id': 20,
  'length': 0}}

In [21]:
# ---------------------------------- readout 1 -------------------------------------
pin_opt_1 = Dict(hfss_wire_bonds = True,
                 pin_inputs=Dict(
                                start_pin=Dict(
                                component='Q_1',
                                pin='readout'),
                                end_pin=Dict(
                                component='readout1_short',
                                pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um'
              )

stg_r1 = ShortToGround(design, 'readout1_short', 
                       options=dict(
                           pos_x='{:f}mm'.format((q_1.pins.readout.points[0][0]+q_1.pins.readout.points[1][0])/2 - 0.560), 
                           pos_y='{:f}mm'.format(q_1.pins.readout.points[0][1]-0.520), orientation='90',
                      width='20um',
                      gap='10um'))

pin_opt_1.pin_inputs.start_pin.component = 'Q_1'
pin_opt_1.pin_inputs.end_pin.component = 'readout1_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_1.lead.start_straight = '520um + 90um'
pin_opt_1.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_1 = OrderedDict()
jogsS_1[0] = ["L", '0um + 180um']
jogsS_1[1] = ["R", '350um + 180um']
jogsS_1[2] = ["R", '200um + 180um']
jogsS_1[3] = ["R", '620um + 180um']
jogsS_1[4] = ["L", '0um + 180um']
jogsS_1[5] = ["L", '420um + 180um']
jogsS_1[6] = ["R", '0um + 160um']
#jogsS_1[7] = ["R", '420um + 90um']

pin_opt_1.lead.start_jogged_extension = jogsS_1
readout_res_1 = RouteFramed(design, 'readout_res_1', pin_opt_1)

# ---------------------------------- readout 2 -------------------------------------
pin_opt_2 = Dict(hfss_wire_bonds = True,
                 pin_inputs=Dict(
                                start_pin=Dict(
                                component='Q_2',
                                pin='readout'),
                                end_pin=Dict(
                                component='readout2_short',
                                pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um'
              )

stg_r2 = ShortToGround(design, 'readout2_short', 
                       options=dict(
                           pos_x='{:f}mm'.format((q_2.pins.readout.points[0][0]+q_2.pins.readout.points[1][0])/2 - 0.560), 
                           pos_y='{:f}mm'.format(q_2.pins.readout.points[0][1] + 0.520), orientation='270',
                      width='20um',
                      gap='10um'))

pin_opt_2.pin_inputs.start_pin.component = 'Q_2'
pin_opt_2.pin_inputs.end_pin.component = 'readout2_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_2.lead.start_straight = '520um + 90um'
pin_opt_2.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_2 = OrderedDict()
jogsS_2[0] = ["R", '0um + 180um']
jogsS_2[1] = ["L", '350um + 180um']
jogsS_2[2] = ["L", '200um + 180um']
jogsS_2[3] = ["L", '620um + 180um']
jogsS_2[4] = ["R", '0um + 180um']
jogsS_2[5] = ["R", '436um + 180um']
jogsS_2[6] = ["L", '0um + 160um']
# jogsS_2 = OrderedDict()
# jogsS_2[0] = ["L", '0um + 180um']
# jogsS_2[1] = ["R", '350um + 180um']
# jogsS_2[2] = ["R", '200um + 180um']
# jogsS_2[3] = ["R", '620um + 180um']
# jogsS_2[4] = ["L", '0um + 180um']
# jogsS_2[5] = ["L", '436um + 180um']
# jogsS_2[6] = ["R", '0um + 160um']
#jogsS_2[7] = ["R", '420um + 90um']

pin_opt_2.lead.start_jogged_extension = jogsS_2
readout_res_2 = RouteFramed(design, 'readout_res_2', pin_opt_2)

gui.rebuild()
gui.autoscale()

# ---------------------------------- Analyze ----------------------------------

# Eigenmode and EPR

In [22]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

In [23]:
eig_qb.sim.renderer.options['wb_size'] = 5

In [32]:
em_p = eig_qb.sim.setup

In [33]:
em_p.name = 'Setup'
em_p.min_freq_ghz = 5.2
em_p.n_modes = 3
em_p.max_passes = 20
em_p.max_delta_f = 0.1
em_p.min_converged = 2
# Design variables can also be added in for direct simulation sweeps.
em_p.vars = Dict({'Lj1': Q1_ind, 'Cj1': '0 fF', 
                  'Lj2': Q2_ind, 'Cj2': '0 fF'})
# em_p.vars = Dict({'Lj1': ['17.2 nH','17.3 nH','17.4 nH','17.5 nH'], 'Cj1': '0 fF', 'Lj2': '17.0 nH', 'Cj2': '0 fF'})
# em_p.vars = Dict({'Lj1': '17.2 nH', 'Cj1': '0 fF'})

eig_qb.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 5.2,
 'n_modes': 3,
 'max_delta_f': 0.1,
 'max_passes': 20,
 'min_passes': 1,
 'min_converged': 2,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj1': '14nH', 'Cj1': '0 fF', 'Lj2': '11nH', 'Cj2': '0 fF'},
 'run': {'name': 'Berkeley_8Qring_sample_3',
  'components': ['Q_1', 'Q_2', 'Bus_12'],
  'open_terminations': [],
  'port_list': None,
  'jj_to_port': None,
  'ignored_jjs': None,
  'box_plus_buffer': True}}

In [29]:
eig_qb.del_junction()
eig_qb.add_junction('jj1', 'Lj1', 'Cj1', rect='JJ_rect_Lj_Q_1_rect_jj', line='JJ_Lj_Q_1_rect_jj_')
eig_qb.add_junction('jj2', 'Lj2', 'Cj2', rect='JJ_rect_Lj_Q_2_rect_jj', line='JJ_Lj_Q_2_rect_jj_')
# eig_qb.add_junction('jj3', 'Lj3', 'Cj3', rect='JJ_rect_Lj_Q_3_rect_jj', line='JJ_Lj_Q_3_rect_jj_')
eig_qb.setup.sweep_variable = 'Lj1'

08:46AM 31s WARNING [add_junction]: junction already defined. Overwriting jj1
08:46AM 31s WARNING [add_junction]: junction already defined. Overwriting jj2


In [34]:
eig_qb.sim.run_sim(name="Berkeley_8Qring_sample_3", components=['Q_1', 'Q_2','Bus_12'], open_terminations=[])
# eig_qb.sim._analyze(design_name = 'Berkeley_8Qring_sample_3_hfss')  # This line dose not update modeling. It remove result, update setup and do analysis
# eig_qb.sim.start() # line for when kernel dies. it connect to open ansys again. you can run epr but cannot plot ansys convergence or things related to simulation.

# all_sweeps, return_code = eig_qb.run_sweep(q_1.name,
#                                            'hfss_inductance',
#                                             Q1_sweep,
#                                             components=['Q_1', 'Q_2','Bus_12'], # empty list means all components
#                                              design_name = 'Berkeley_8Qring_sample_3'
#                                              )

# resonator induced frequency shift
# all_sweeps, return_code = eig_qb.run_sweep(q_2.name,
#                                            'hfss_inductance',
#                                             Q1_sweep,
#                                             components= ['Q_1','Q_2','Q_3','Bus_12','Bus_23','readout_res_2','readout2_short'], # empty list means all components
#                                              design_name = 'Berkeley_8Qring_sample_3'
#                                              )

com_error: (-2147417851, '서버에서 예외 오류가 발생했습니다.', None, None)

In [28]:
eig_qb.sim.run_sim(name="Berkeley_8Qring_sample_3_S21", components=[], open_terminations=[])

INFO 04:13PM [connect_design]: 	Opened active design
	Design:    Berkeley_8Qring_sample_3_S21_hfss [Solution type: Eigenmode]
WARNING 04:13PM [connect_setup]: 	No design setup detected.
WARNING 04:13PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:13PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:14PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:14PM [analyze]: Analyzing setup Setup
04:14PM 50s INFO [get_f_convergence]: Saved convergences to c:\Users\user\Documents\GitHub\qiskit-metal\hfss_eig_f_convergence.csv


('Berkeley_8Qring_sample_3_S21_hfss', 'Setup')

In [88]:
# time.sleep(2)
eig_qb.sim.plot_convergences()

TypeError: 'Dict' object is not callable

In [29]:
eig_qb.setup
# rend = eig_qb.sim.renderer
# eig_qb.setup.setmode([1,2,6])

{'junctions': {'jj1': {'Lj_variable': 'Lj1',
   'Cj_variable': 'Cj1',
   'rect': 'JJ_rect_Lj_Q_1_rect_jj',
   'line': 'JJ_Lj_Q_1_rect_jj_'},
  'jj2': {'Lj_variable': 'Lj2',
   'Cj_variable': 'Cj2',
   'rect': 'JJ_rect_Lj_Q_2_rect_jj',
   'line': 'JJ_Lj_Q_2_rect_jj_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj1'}

In [30]:

eig_qb.run_epr()
# (pyEPR allows to switch modes: eprd.set_mode(1))

Design "Berkeley_8Qring_sample_3_hfss" info:
	# eigenmodes    3
	# variations    1
Design "Berkeley_8Qring_sample_3_hfss" info:
	# eigenmodes    3
	# variations    1

        energy_elec_all       = 9.28987080803236e-25
        energy_elec_substrate = 8.55736247564873e-25
        EPR of substrate = 92.1%

        energy_mag    = 9.27519864933177e-25
        energy_mag % of energy_elec_all  = 99.8%
        

Variation 0  [1/1]

  Mode 0 at 5.56 GHz   [1/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.0%   1.69e-25 1.638e-27

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 97.52%
	jj1             0.000158749  (+)        4.03127e-06
		Energy fraction (Lj over Lj&Cj)= 97.41%
	jj2             0.989475  (+)        0.0262622
		(U_tot_cap-U_tot_ind)/mean=1.33%
Calculating Qdielectric_main for mode 0 (0

WARNING 11:32AM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Trailblazer_test\Berkeley_8Qring_sample_3_hfss\2024-07-16 11-30-45.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.027098
1    1.027170
2    1.027846
dtype: float64



  result['Q_coupling'] = self.Qm_coupling[variation][self.Qm_coupling[variation].columns[junctions]][modes]#TODO change the columns to junctions

  result['Qs'] = self.Qs[variation][self.PM[variation].columns[junctions]][modes] #TODO change the columns to junctions



Pm_norm idx =
     jj1    jj2
0  False   True
1   True  False
2  False  False
*** P (participation matrix, not normlz.)
        jj1       jj2
0  0.000155  0.964150
1  0.963622  0.000173
2  0.000839  0.000740

*** S (sign-bit matrix)
   s_jj1  s_jj2
0      1     -1
1     -1     -1
2     -1      1
*** P (participation matrix, normalized.)
   0.00015     0.99
      0.99  0.00017
   0.00084  0.00074

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       252    0.167    0.549
     0.167      252    0.609
     0.549    0.609 0.000666

*** Chi matrix ND (MHz) 
       279  -0.0299    0.458
   -0.0299      278      0.5
     0.458      0.5 0.000468

*** Frequencies O1 PT (MHz)
0    5308.571143
1    5432.890785
2    8103.271518
dtype: float64

*** Frequencies ND (MHz)
0    5295.830624
1    5420.484859
2    8103.300628
dtype: float64

*** Q_coupling
Empty DataFrame
Columns: []
Index: [0, 1, 2]


...

###### Numerical diagonalization

Lj1,10.4
0,5295.83
1,5420.48
2,8103.30


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0       1         2
Lj1                             
10.4 0  279.10   -0.03  4.58e-01
     1   -0.03  278.27  5.00e-01
     2    0.46    0.50  4.68e-04

In [29]:
eig_qb.sim.close()

# 360

In [ ]:
Phi_0 = 2.067833848e-15
Lj_test = 17
Ic_test = Phi_0/2/np.pi/Lj_test/1e-9
print("When Lj={:.3f}[nH],".format(Lj_test)+"Ic={:.3f}[nA]".format(Ic_test/1e-9))

When Lj=17.000[nH],Ic=19.359[nA]
